In [1]:
import os
import numpy as np

# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.tri 
# import mpl_toolkits.mplot3d
# My conda does not recognise this ???


from rdkit import Chem
from rdkit.Chem import Draw
import rdkit.Chem.AllChem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors

from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import make_union, Pipeline
from collections import Counter

from gtda.mapper import plot_interactive_mapper_graph
from gtda.plotting import plot_point_cloud, plot_betti_curves
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy
from gtda.diagrams import NumberOfPoints
from gtda.diagrams import Amplitude

import structures as st # My own module

In [2]:
def generate_structure_from_smiles(smiles): #This works for rdkit (organic materials), not really crystal structures
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    status = AllChem.EmbedMolecule(mol)
    status = AllChem.UFFOptimizeMolecule(mol)

    conformer = mol.GetConformer()
    coordinates = conformer.GetPositions()
    coordinates = np.array(coordinates)

    return coordinates 

def smiles_to_persistence_diagrams(smiles):
    coords=generate_structure_from_smiles(smiles)
    # Track connected components, loops, and voids
    homology_dimensions = [0, 1, 2]

    # Look into this?
    persistence = VietorisRipsPersistence(
        metric="euclidean",
        homology_dimensions=homology_dimensions,
        n_jobs=1,
        collapse_edges=True,
    )
    reshaped_coords=coords[None, :, :]
    diagrams_basic = persistence.fit_transform(reshaped_coords)  #error occurs here when the njobs > 1
    return coords, diagrams_basic

def persistence_diagrams(coords):
    # Track connected components, loops, and voids
    homology_dimensions = [0, 1, 2]

    # Collapse edges to speed up H2 persistence calculation!
    persistence = VietorisRipsPersistence(
        metric="euclidean",
        homology_dimensions=homology_dimensions,
        n_jobs=1,
        collapse_edges=True,
    )
    
    reshaped_coords=coords[None, :, :]
    diagrams_basic = persistence.fit_transform(reshaped_coords)
    return coords, diagrams_basic

In [3]:
"""Persistent-homology–related plotting functions and classes."""
# License: GNU AGPLv3

import numpy as np
import plotly.graph_objs as gobj


def plot_diagram(diagram, homology_dimensions=None, plotly_params=None):
    """Plot a single persistence diagram.

    Parameters
    ----------
    diagram : ndarray of shape (n_points, 3)
        The persistence diagram to plot, where the third dimension along axis 1
        contains homology dimensions, and the first two contain (birth, death)
        pairs to be used as coordinates in the two-dimensional plot.

    homology_dimensions : list of int or None, optional, default: ``None``
        Homology dimensions which will appear on the plot. If ``None``, all
        homology dimensions which appear in `diagram` will be plotted.

    plotly_params : dict or None, optional, default: ``None``
        Custom parameters to configure the plotly figure. Allowed keys are
        ``"traces"`` and ``"layout"``, and the corresponding values should be
        dictionaries containing keyword arguments as would be fed to the
        :meth:`update_traces` and :meth:`update_layout` methods of
        :class:`plotly.graph_objects.Figure`.

    Returns
    -------
    fig : :class:`plotly.graph_objects.Figure` object
        Figure representing the persistence diagram.

    """
    # TODO: increase the marker size
    if homology_dimensions is None:
        homology_dimensions = np.unique(diagram[:, 2])

    diagram = diagram[diagram[:, 0] != diagram[:, 1]]
    diagram_no_dims = diagram[:, :2]
    posinfinite_mask = np.isposinf(diagram_no_dims)
    neginfinite_mask = np.isneginf(diagram_no_dims)
    if diagram_no_dims.size:
        max_val = np.max(np.where(posinfinite_mask, -np.inf, diagram_no_dims))
        min_val = np.min(np.where(neginfinite_mask, np.inf, diagram_no_dims))
    else:
        # Dummy values if diagram is empty
        max_val = 1
        min_val = 0
    parameter_range = max_val - min_val
    extra_space_factor = 0.02
    has_posinfinite_death = np.any(posinfinite_mask[:, 1])
    if has_posinfinite_death:
        posinfinity_val = max_val + 0.1 * parameter_range
        extra_space_factor += 0.1
    extra_space = extra_space_factor * parameter_range
    min_val_display = min_val - extra_space
    max_val_display = max_val + extra_space

    fig = gobj.Figure()
    fig.add_trace(gobj.Scatter(
        x=[min_val_display, max_val_display],
        y=[min_val_display, max_val_display],
        mode="lines",
        line={"dash": "dash", "width": 1, "color": "black"},
        showlegend=False,
        hoverinfo="none"
        ))

    for dim in homology_dimensions:
        name = f"H{int(dim)}" if dim != np.inf else "Any homology dimension"
        subdiagram = diagram[diagram[:, 2] == dim]
        unique, inverse, counts = np.unique(
            subdiagram, axis=0, return_inverse=True, return_counts=True
            )
        hovertext = [
            f"{tuple(unique[unique_row_index][:2])}" +
            (
                f", multiplicity: {counts[unique_row_index]}"
                if counts[unique_row_index] > 1 else ""
            )
            for unique_row_index in inverse
            ]
        y = subdiagram[:, 1]
        if has_posinfinite_death:
            y[np.isposinf(y)] = posinfinity_val
        fig.add_trace(gobj.Scatter(
            x=subdiagram[:, 0], y=y, mode="markers + text", textposition="top center",
            hoverinfo="text", hovertext=hovertext, name=name
        ))

    fig.update_layout(
        width=500,
        height=500,
        xaxis1={
            "title": "Birth",
            "side": "bottom",
            "type": "linear",
            "range": [min_val_display, max_val_display],
            "autorange": False,
            "ticks": "outside",
            "showline": True,
            "zeroline": True,
            "linewidth": 1,
            "linecolor": "black",
            "mirror": False,
            "showexponent": "all",
            "exponentformat": "e"
            },
        yaxis1={
            "title": "Death",
            "side": "left",
            "type": "linear",
            "range": [min_val_display, max_val_display],
            "autorange": False, "scaleanchor": "x", "scaleratio": 1,
            "ticks": "outside",
            "showline": True,
            "zeroline": True,
            "linewidth": 1,
            "linecolor": "black",
            "mirror": False,
            "showexponent": "all",
            "exponentformat": "e"
            },
        plot_bgcolor="white"
        )

    # Add a horizontal dashed line for points with infinite death
    if has_posinfinite_death:
        fig.add_trace(gobj.Scatter(
            x=[min_val_display, max_val_display],
            y=[posinfinity_val, posinfinity_val],
            mode="lines",
            line={"dash": "dash", "width": 0.5, "color": "black"},
            showlegend=True,
            name=u"\u221E",
            hoverinfo="none"
        ))

    # Update traces and layout according to user input
    if plotly_params:
        fig.update_traces(plotly_params.get("traces", None))
        fig.update_layout(plotly_params.get("layout", None))

    return fig

In [4]:
simple_cubic = st.Structure(1.0, 1.0, 1.0, 2, 2, 2, 90, 90, 90, False, False, False)
body_centered_cubic = st.Structure(1.0, 1.0, 1.0, 2, 2, 2, 90, 90, 90, False, False, True)
face_centered_cubic = st.Structure(1.0, 1.0, 1.0, 2, 2, 2, 90, 90, 90, True, True, False)

simple_tetragonal = st.Structure(1.0, 1.0, 1.5, 2, 2, 2, 90, 90, 90, False, False, False)
body_centered_tetragonal = st.Structure(1.0, 1.0, 1.5, 2, 2, 2, 90, 90, 90, False, False, True)

simple_orthorhombic = st.Structure(1.0, 1.25, 1.5, 2, 2, 2, 90, 90, 90, False, False, False)
body_centered_orothorhombic = st.Structure(1.0, 1.25, 1.5, 2, 2, 2, 90, 90, 90, False, False, True)
base_centered_orthorhombic = st.Structure(1.0, 1.25, 1.5, 2, 2, 2, 90, 90, 90, True, False, False)
face_centered_orthorhombic = st.Structure(1.0, 1.25, 1.5, 2, 2, 2, 90, 90, 90, True, True, False)

simple_monoclinic = st.Structure(1.0, 1.25, 1.5, 2, 2, 2, 90, 80, 90, False, False, False)
base_centered_monoclinic = st.Structure(1.0, 1.25, 1.5, 2, 2, 2, 90, 80, 90, False, True, False)

hexagonal = st.Structure(1.0, 1.0, 1.25, 2, 2, 2, 90, 90, 120, False, False, False)
rhombohedral = st.Structure(1.0, 1.0, 1.0, 2, 2, 2, 80, 80, 80, False, False, False)
triclinic = st.Structure(1.0, 1.25, 1.5, 2, 2, 2, 80, 70, 60, False, False, False)

In [5]:
for structure in [simple_cubic, body_centered_cubic, face_centered_cubic, simple_tetragonal, body_centered_tetragonal, simple_orthorhombic, body_centered_orothorhombic, base_centered_orthorhombic, face_centered_orthorhombic, simple_monoclinic, base_centered_monoclinic, triclinic, hexagonal, rhombohedral]:
    coords, diagrams_basic = persistence_diagrams(structure)
    figure = plot_diagram(diagrams_basic[0])
    figure.show()